In [1]:
from transformers import AutoConfig,AutoModel,AutoTokenizer
import torch
import os

In [2]:
model_src = r"/root/autodl-tmp/chatglm2-6b-int4"
CHECKPOINT_PATH = r"/root/autodl-tmp/fix-helper/LLM-langchain/LLM-train/ptuning/output/adgen-chatglm2-6b-pt-128-2e-2/checkpoint-1000"
tokenizer = AutoTokenizer.from_pretrained(model_src,trust_remote_code=True)

config = AutoConfig.from_pretrained(model_src, trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained(model_src, config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)



Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at /root/autodl-tmp/chatglm2-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
# Comment out the following line if you don't use quantization
model = model.quantize(4)
model = model.cuda()
model = model.eval() 

In [6]:
response, history = model.chat(tokenizer, "My Samsung laptop's touchpad is not working properly, what should I do?", history=[])

print(response)

If the touchpad is not responding, you should first check if the touchpad driver is installed correctly through the settings. Open settings and look for 'Samsung Update,' then find 'Touchpad Driver' to ensure the touchpad is installed. If installed, it will show ' installed' in the text menu. If that doesn't work, try using a keyboard or touchpad with a finger to test the touchpad's touch sensitivity. If it feels hard to touch, it may be faulty or defective and need to replace.
